<a href="https://colab.research.google.com/github/anquabkhan/google-collab/blob/main/Space_Missions_Analysis_(start).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

<center><img src="https://i.imgur.com/9hLRsjZ.jpg" height=400></center>

This dataset was scraped from [nextspaceflight.com](https://nextspaceflight.com/launches/past/?page=1) and includes all the space missions since the beginning of Space Race between the USA and the Soviet Union in 1957!

### Install Package with Country Codes

In [1]:
%pip install iso3166

### Upgrade Plotly

Run the cell below if you are working with Google Colab.

In [2]:
%pip install --upgrade plotly

     |████████████████████████████████| 23.9 MB 13 kB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


### Import Statements

In [3]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# These might be helpful:

from iso3166 import countries
from datetime import datetime, timedelta

### Notebook Presentation

In [4]:
pd.options.display.float_format = '{:,.2f}'.format

### Load the Data

In [6]:
df_data = pd.read_csv('/content/data/mission_launches.csv')

# Preliminary Data Exploration

* What is the shape of `df_data`? 
* How many rows and columns does it have?
* What are the column names?
* Are there any NaN values or duplicates?

In [7]:
print(df_data.shape)
print(df_data.columns)
print(df_data.isna().values.any())
print(df_data.duplicated().values.any())




(4324, 9)
Index(['Unnamed: 0', 'Unnamed: 0.1', 'Organisation', 'Location', 'Date',
       'Detail', 'Rocket_Status', 'Price', 'Mission_Status'],
      dtype='object')
True
False


In [8]:
df_data

,Unnamed: 0,Unnamed: 0.1,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success
...,...,...,...,...,...,...,...,...,...
4319,4319,4319,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA","Wed Feb 05, 1958 07:33 UTC",Vanguard | Vanguard TV3BU,StatusRetired,NaN,Failure
4320,4320,4320,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA","Sat Feb 01, 1958 03:48 UTC",Juno I | Explorer 1,StatusRetired,NaN,Success
4321,4321,4321,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA","Fri Dec 06, 1957 16:44 UTC",Vanguard | Vanguard TV3,StatusRetired,NaN,Failure
4322,4322,4322,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan","Sun Nov 03, 1957 02:30 UTC",Sputnik 8K71PS | Sputnik-2,StatusRetired,NaN,Success


## Data Cleaning - Check for Missing Values and Duplicates

Consider removing columns containing junk data. 

In [9]:
df_data2 = df_data.dropna(axis='columns')

## Descriptive Statistics

In [10]:
df_data2.describe()

,Unnamed: 0,Unnamed: 0.1
count,"4,324.00","4,324.00"
mean,"2,161.50","2,161.50"
std,"1,248.38","1,248.38"
min,0.00,0.00
25%,"1,080.75","1,080.75"
50%,"2,161.50","2,161.50"
75%,"3,242.25","3,242.25"
max,"4,323.00","4,323.00"


In [11]:
print(df_data2.describe())
df_data2.info()

       Unnamed: 0  Unnamed: 0.1
count    4,324.00      4,324.00
mean     2,161.50      2,161.50
std      1,248.38      1,248.38
min          0.00          0.00
25%      1,080.75      1,080.75
50%      2,161.50      2,161.50
75%      3,242.25      3,242.25
max      4,323.00      4,323.00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4324 entries, 0 to 4323
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      4324 non-null   int64 
 1   Unnamed: 0.1    4324 non-null   int64 
 2   Organisation    4324 non-null   object
 3   Location        4324 non-null   object
 4   Date            4324 non-null   object
 5   Detail          4324 non-null   object
 6   Rocket_Status   4324 non-null   object
 7   Mission_Status  4324 non-null   object
dtypes: int64(2), object(6)
memory usage: 270.4+ KB


# Number of Launches per Company

Create a chart that shows the number of space mission launches by organisation.

In [12]:
org_count = df_data2.value_counts('Organisation')
org_count

Organisation
RVSN USSR           1777
Arianespace          279
General Dynamics     251
CASC                 251
NASA                 203
VKS RF               201
US Air Force         161
ULA                  140
Boeing               136
Martin Marietta      114
SpaceX               100
MHI                   84
Northrop              83
Lockheed              79
ISRO                  76
Roscosmos             55
ILS                   46
Sea Launch            36
ISAS                  30
Kosmotras             22
US Navy               17
Eurockot              13
ESA                   13
ISA                   13
Rocket Lab            13
Blue Origin           12
IAI                   11
ExPace                10
ASI                    9
CNES                   8
AMBA                   8
JAXA                   7
Land Launch            7
MITT                   7
KCST                   5
UT                     5
CASIC                  5
CECLES                 4
Arm??e de l'Air        4
Exos        

In [13]:
fig = px.bar(data_frame=org_count, x=org_count.index, y=org_count.values, labels={ "org_count.index": "Number of launches"},height=500 )
fig.update_layout(yaxis_title = "Number of launches")



# Number of Active versus Retired Rockets

How many rockets are active compared to those that are decomissioned? 

In [14]:
df_data2

,Unnamed: 0,Unnamed: 0.1,Organisation,Location,Date,Detail,Rocket_Status,Mission_Status
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,Success
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,Success
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,Success
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,Success
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,Success
...,...,...,...,...,...,...,...,...
4319,4319,4319,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA","Wed Feb 05, 1958 07:33 UTC",Vanguard | Vanguard TV3BU,StatusRetired,Failure
4320,4320,4320,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA","Sat Feb 01, 1958 03:48 UTC",Juno I | Explorer 1,StatusRetired,Success
4321,4321,4321,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA","Fri Dec 06, 1957 16:44 UTC",Vanguard | Vanguard TV3,StatusRetired,Failure
4322,4322,4322,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan","Sun Nov 03, 1957 02:30 UTC",Sputnik 8K71PS | Sputnik-2,StatusRetired,Success


In [15]:
df_data2.value_counts('Rocket_Status')

Rocket_Status
StatusRetired    3534
StatusActive      790
dtype: int64

# Distribution of Mission Status

How many missions were successful?
How many missions failed?

In [16]:
df_data2.value_counts('Mission_Status')

Mission_Status
Success              3879
Failure               339
Partial Failure       102
Prelaunch Failure       4
dtype: int64

# How Expensive are the Launches? 

Create a histogram and visualise the distribution. The price column is given in USD millions (careful of missing values). 

In [17]:
df_data

,Unnamed: 0,Unnamed: 0.1,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success
...,...,...,...,...,...,...,...,...,...
4319,4319,4319,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA","Wed Feb 05, 1958 07:33 UTC",Vanguard | Vanguard TV3BU,StatusRetired,NaN,Failure
4320,4320,4320,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA","Sat Feb 01, 1958 03:48 UTC",Juno I | Explorer 1,StatusRetired,NaN,Success
4321,4321,4321,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA","Fri Dec 06, 1957 16:44 UTC",Vanguard | Vanguard TV3,StatusRetired,NaN,Failure
4322,4322,4322,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan","Sun Nov 03, 1957 02:30 UTC",Sputnik 8K71PS | Sputnik-2,StatusRetired,NaN,Success


In [18]:
px.histogram(data_frame=df_data, x=df_data['Organisation'])

# Use a Choropleth Map to Show the Number of Launches by Country

* Create a choropleth map using [the plotly documentation](https://plotly.com/python/choropleth-maps/)
* Experiment with [plotly's available colours](https://plotly.com/python/builtin-colorscales/). I quite like the sequential colour `matter` on this map. 
* You'll need to extract a `country` feature as well as change the country names that no longer exist.

Wrangle the Country Names

You'll need to use a 3 letter country code for each country. You might have to change some country names.

* Russia is the Russian Federation
* New Mexico should be USA
* Yellow Sea refers to China
* Shahrud Missile Test Site should be Iran
* Pacific Missile Range Facility should be USA
* Barents Sea should be Russian Federation
* Gran Canaria should be USA


You can use the iso3166 package to convert the country names to Alpha3 format.

In [19]:
my_df = pd.DataFrame(data={
    'coutries': ['usa','india','france','germany'],
    'price': [100, 200, 300, 400]
})
df_data

,Unnamed: 0,Unnamed: 0.1,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success
...,...,...,...,...,...,...,...,...,...
4319,4319,4319,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA","Wed Feb 05, 1958 07:33 UTC",Vanguard | Vanguard TV3BU,StatusRetired,NaN,Failure
4320,4320,4320,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA","Sat Feb 01, 1958 03:48 UTC",Juno I | Explorer 1,StatusRetired,NaN,Success
4321,4321,4321,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA","Fri Dec 06, 1957 16:44 UTC",Vanguard | Vanguard TV3,StatusRetired,NaN,Failure
4322,4322,4322,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan","Sun Nov 03, 1957 02:30 UTC",Sputnik 8K71PS | Sputnik-2,StatusRetired,NaN,Success


In [22]:
i = 0 
for x in df_data['Location'].str.split(', '):
  df_data['nations'][i] = x[-1] 
  i += 1
# df_data 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [23]:
df_data['nations'].replace({'Russia': 'Russian Federation', 'New Mexico': 'USA','Yellow Sea': 'China','Shahrud Missile Test Site': 'Iraq',
 'Pacific Missile Range Facility': 'USA','Barents Sea': 'Russian Federation', 'Gran Canaria': 'USA', 'Zealand': 'new zealand', 'Site':'Iraq', 'North Korea':'Korea, Republic of',  
  'Iran': 'Iraq', 'Pacific Ocean': 'India', 'South Korea':'Korea, Republic of'}, inplace=True)

In [26]:
i=0
for x in df_data['nations']:
  df_data['country_codes'][i] = countries.get(x).alpha3
  i += 1 
countries.get('Iraq') 
df_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Unnamed: 0,Unnamed: 0.1,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status,nations,country_codes
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success,USA,USA
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success,China,CHN
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success,USA,USA
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success,Kazakhstan,KAZ
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success,USA,USA
...,...,...,...,...,...,...,...,...,...,...,...
4319,4319,4319,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA","Wed Feb 05, 1958 07:33 UTC",Vanguard | Vanguard TV3BU,StatusRetired,NaN,Failure,USA,USA
4320,4320,4320,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA","Sat Feb 01, 1958 03:48 UTC",Juno I | Explorer 1,StatusRetired,NaN,Success,USA,USA
4321,4321,4321,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA","Fri Dec 06, 1957 16:44 UTC",Vanguard | Vanguard TV3,StatusRetired,NaN,Failure,USA,USA
4322,4322,4322,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan","Sun Nov 03, 1957 02:30 UTC",Sputnik 8K71PS | Sputnik-2,StatusRetired,NaN,Success,Kazakhstan,KAZ


In [27]:
df_data['nations'].sample(60)
df_data[df_data['nations']=='Pacific Ocean']

,Unnamed: 0,Unnamed: 0.1,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status,nations,country_codes


In [28]:
# i = 0
# for x in df_data['Location'] :
#   y = x.split(',')
#   df_data['nations'][i] = y[-1]
#   i += 1 
# df_data 
for c in countries:
  print(c) 
# countries.get('Korea')

Country(name='Afghanistan', alpha2='AF', alpha3='AFG', numeric='004', apolitical_name='Afghanistan')
Country(name='Åland Islands', alpha2='AX', alpha3='ALA', numeric='248', apolitical_name='Åland Islands')
Country(name='Albania', alpha2='AL', alpha3='ALB', numeric='008', apolitical_name='Albania')
Country(name='Algeria', alpha2='DZ', alpha3='DZA', numeric='012', apolitical_name='Algeria')
Country(name='American Samoa', alpha2='AS', alpha3='ASM', numeric='016', apolitical_name='American Samoa')
Country(name='Andorra', alpha2='AD', alpha3='AND', numeric='020', apolitical_name='Andorra')
Country(name='Angola', alpha2='AO', alpha3='AGO', numeric='024', apolitical_name='Angola')
Country(name='Anguilla', alpha2='AI', alpha3='AIA', numeric='660', apolitical_name='Anguilla')
Country(name='Antarctica', alpha2='AQ', alpha3='ATA', numeric='010', apolitical_name='Antarctica')
Country(name='Antigua and Barbuda', alpha2='AG', alpha3='ATG', numeric='028', apolitical_name='Antigua and Barbuda')
Countr

In [29]:
my_df

,coutries,price
0,usa,100
1,india,200
2,france,300
3,germany,400


In [30]:
df_data.columns
df_data['Price'] = df_data['Price'].str.replace(',','')
df_data['Price']= df_data['Price'].fillna(0)
df_data['Price'].sample(50)
df_data['Price'] = pd.to_numeric(df_data['Price'])

In [31]:
grp_data = df_data.groupby('country_codes').sum()

In [32]:
fig = px.choropleth(grp_data, scope='world', color=grp_data['Price'] ,color_continuous_scale='matter', locationmode='ISO-3', locations= grp_data.index)
# fig. = px.colors.sequential.swatches_continuous()
# plotly.express.colors.diverging color scales as the inputs to
# color_continuous_scale.
fig.show()

# Use a Choropleth Map to Show the Number of Failures by Country


In [33]:
df_data.info()
df_data3 = df_data[df_data['Mission_Status'] == 'Failure']
counts_df = df_data.groupby('country_codes').count()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4324 entries, 0 to 4323
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      4324 non-null   int64  
 1   Unnamed: 0.1    4324 non-null   int64  
 2   Organisation    4324 non-null   object 
 3   Location        4324 non-null   object 
 4   Date            4324 non-null   object 
 5   Detail          4324 non-null   object 
 6   Rocket_Status   4324 non-null   object 
 7   Price           4324 non-null   float64
 8   Mission_Status  4324 non-null   object 
 9   nations         4324 non-null   object 
 10  country_codes   4324 non-null   object 
dtypes: float64(1), int64(2), object(8)
memory usage: 371.7+ KB


In [34]:
counts_df.rename(columns={'Mission_Status': 'Number of failed launches'}, inplace=True)

In [35]:
px.choropleth(data_frame=counts_df, locations=counts_df.index, color=counts_df['Number of failed launches'], color_continuous_scale='matter')

In [36]:
df_data

,Unnamed: 0,Unnamed: 0.1,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status,nations,country_codes
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.00,Success,USA,USA
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success,China,CHN
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,0.00,Success,USA,USA
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.00,Success,Kazakhstan,KAZ
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.00,Success,USA,USA
...,...,...,...,...,...,...,...,...,...,...,...
4319,4319,4319,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA","Wed Feb 05, 1958 07:33 UTC",Vanguard | Vanguard TV3BU,StatusRetired,0.00,Failure,USA,USA
4320,4320,4320,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA","Sat Feb 01, 1958 03:48 UTC",Juno I | Explorer 1,StatusRetired,0.00,Success,USA,USA
4321,4321,4321,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA","Fri Dec 06, 1957 16:44 UTC",Vanguard | Vanguard TV3,StatusRetired,0.00,Failure,USA,USA
4322,4322,4322,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan","Sun Nov 03, 1957 02:30 UTC",Sputnik 8K71PS | Sputnik-2,StatusRetired,0.00,Success,Kazakhstan,KAZ


# Create a Plotly Sunburst Chart of the countries, organisations, and mission status. 

In [37]:
df_data

,Unnamed: 0,Unnamed: 0.1,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status,nations,country_codes
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.00,Success,USA,USA
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success,China,CHN
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,0.00,Success,USA,USA
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.00,Success,Kazakhstan,KAZ
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.00,Success,USA,USA
...,...,...,...,...,...,...,...,...,...,...,...
4319,4319,4319,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA","Wed Feb 05, 1958 07:33 UTC",Vanguard | Vanguard TV3BU,StatusRetired,0.00,Failure,USA,USA
4320,4320,4320,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA","Sat Feb 01, 1958 03:48 UTC",Juno I | Explorer 1,StatusRetired,0.00,Success,USA,USA
4321,4321,4321,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA","Fri Dec 06, 1957 16:44 UTC",Vanguard | Vanguard TV3,StatusRetired,0.00,Failure,USA,USA
4322,4322,4322,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan","Sun Nov 03, 1957 02:30 UTC",Sputnik 8K71PS | Sputnik-2,StatusRetired,0.00,Success,Kazakhstan,KAZ


In [38]:
fig = px.sunburst(df_data, path=['nations', 'Organisation', 'Mission_Status'], values=df_data['Price'])  

In [39]:
fig.show() 

# Analyse the Total Amount of Money Spent by Organisation on Space Missions

In [40]:
df_data

,Unnamed: 0,Unnamed: 0.1,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status,nations,country_codes
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.00,Success,USA,USA
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success,China,CHN
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,0.00,Success,USA,USA
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.00,Success,Kazakhstan,KAZ
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.00,Success,USA,USA
...,...,...,...,...,...,...,...,...,...,...,...
4319,4319,4319,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA","Wed Feb 05, 1958 07:33 UTC",Vanguard | Vanguard TV3BU,StatusRetired,0.00,Failure,USA,USA
4320,4320,4320,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA","Sat Feb 01, 1958 03:48 UTC",Juno I | Explorer 1,StatusRetired,0.00,Success,USA,USA
4321,4321,4321,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA","Fri Dec 06, 1957 16:44 UTC",Vanguard | Vanguard TV3,StatusRetired,0.00,Failure,USA,USA
4322,4322,4322,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan","Sun Nov 03, 1957 02:30 UTC",Sputnik 8K71PS | Sputnik-2,StatusRetired,0.00,Success,Kazakhstan,KAZ


# Analyse the Amount of Money Spent by Organisation per Launch

In [41]:
df_data

,Unnamed: 0,Unnamed: 0.1,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status,nations,country_codes
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.00,Success,USA,USA
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success,China,CHN
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,0.00,Success,USA,USA
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.00,Success,Kazakhstan,KAZ
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.00,Success,USA,USA
...,...,...,...,...,...,...,...,...,...,...,...
4319,4319,4319,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA","Wed Feb 05, 1958 07:33 UTC",Vanguard | Vanguard TV3BU,StatusRetired,0.00,Failure,USA,USA
4320,4320,4320,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA","Sat Feb 01, 1958 03:48 UTC",Juno I | Explorer 1,StatusRetired,0.00,Success,USA,USA
4321,4321,4321,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA","Fri Dec 06, 1957 16:44 UTC",Vanguard | Vanguard TV3,StatusRetired,0.00,Failure,USA,USA
4322,4322,4322,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan","Sun Nov 03, 1957 02:30 UTC",Sputnik 8K71PS | Sputnik-2,StatusRetired,0.00,Success,Kazakhstan,KAZ


# Chart the Number of Launches per Year

In [42]:
df_data['Date'] = pd.to_datetime(df_data.Date, utc=True)
# print(df_data.info())
# df_data['Date'] = pd.DatetimeIndex(df_data['Date'])
if isinstance(df_data['Date'], pd.DatetimeIndex):
    print('Success')
else:
  print('failiure')
# print(pd.DatetimeIndex(df_data['Date']))
d = pd.DatetimeIndex( df_data['Date'],dtype='datetime64[ns, UTC]') 


failiure


In [43]:

# new_df = df_data.resample('Y', on='Date')
# new_df.bfill
# new_df.dropna()
# new_df['country_codes'] = df_data.resample('Y', on='Date')
# df.agg({'A' : ['sum', 'min'], 'B' : ['min', 'max']})
# df_x.resample('5Min').agg({'price': 'mean', 'vol': 'sum'}).head()
# n_df = df_data 
# new_df = df_data.resample('Y', on='Date').agg({'country_codes': 'bfill', 'Organisation': 'bfill','Price': 'sum',})
# new_grp = new_df.groupby(['country_codes','Organisation'], as_index=False)['Price']
# new_grp.sum()
# new_df = df_data.resample('Y', on='Date').agg({'country_codes': 'bfill', 'Organisation': 'bfill', 'Price': 'sum')
df_data




,Unnamed: 0,Unnamed: 0.1,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status,nations,country_codes
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA",2020-08-07 05:12:00+00:00,Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.00,Success,USA,USA
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...",2020-08-06 04:01:00+00:00,Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success,China,CHN
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA",2020-08-04 23:57:00+00:00,Starship Prototype | 150 Meter Hop,StatusActive,0.00,Success,USA,USA
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan",2020-07-30 21:25:00+00:00,Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.00,Success,Kazakhstan,KAZ
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA",2020-07-30 11:50:00+00:00,Atlas V 541 | Perseverance,StatusActive,145.00,Success,USA,USA
...,...,...,...,...,...,...,...,...,...,...,...
4319,4319,4319,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1958-02-05 07:33:00+00:00,Vanguard | Vanguard TV3BU,StatusRetired,0.00,Failure,USA,USA
4320,4320,4320,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA",1958-02-01 03:48:00+00:00,Juno I | Explorer 1,StatusRetired,0.00,Success,USA,USA
4321,4321,4321,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1957-12-06 16:44:00+00:00,Vanguard | Vanguard TV3,StatusRetired,0.00,Failure,USA,USA
4322,4322,4322,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",1957-11-03 02:30:00+00:00,Sputnik 8K71PS | Sputnik-2,StatusRetired,0.00,Success,Kazakhstan,KAZ


In [47]:
for i in range(0, len( df_data.Date)):
  df_data['Year'][i]= df_data['Date'][i].year 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [48]:
df_data

,Unnamed: 0,Unnamed: 0.1,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status,nations,country_codes,Year
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA",2020-08-07 05:12:00+00:00,Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.00,Success,USA,USA,2020
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...",2020-08-06 04:01:00+00:00,Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success,China,CHN,2020
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA",2020-08-04 23:57:00+00:00,Starship Prototype | 150 Meter Hop,StatusActive,0.00,Success,USA,USA,2020
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan",2020-07-30 21:25:00+00:00,Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.00,Success,Kazakhstan,KAZ,2020
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA",2020-07-30 11:50:00+00:00,Atlas V 541 | Perseverance,StatusActive,145.00,Success,USA,USA,2020
...,...,...,...,...,...,...,...,...,...,...,...,...
4319,4319,4319,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1958-02-05 07:33:00+00:00,Vanguard | Vanguard TV3BU,StatusRetired,0.00,Failure,USA,USA,1958
4320,4320,4320,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA",1958-02-01 03:48:00+00:00,Juno I | Explorer 1,StatusRetired,0.00,Success,USA,USA,1958
4321,4321,4321,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1957-12-06 16:44:00+00:00,Vanguard | Vanguard TV3,StatusRetired,0.00,Failure,USA,USA,1957
4322,4322,4322,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",1957-11-03 02:30:00+00:00,Sputnik 8K71PS | Sputnik-2,StatusRetired,0.00,Success,Kazakhstan,KAZ,1957


In [49]:
grp_d = df_data.groupby(['country_codes', 'Organisation','Year'], as_index=False).agg({'Price': 'sum'})

In [50]:
grp_d

,country_codes,Organisation,Year,Price
0,AUS,AMBA,1967,0.00
1,AUS,CECLES,1968,0.00
2,AUS,CECLES,1969,0.00
3,AUS,CECLES,1970,0.00
4,AUS,RAE,1970,0.00
...,...,...,...,...
719,USA,US Air Force,1984,0.00
720,USA,US Navy,1957,0.00
721,USA,US Navy,1958,0.00
722,USA,US Navy,1959,0.00


In [51]:
px.sunburst(grp_d, path=[grp_d['country_codes'], grp_d['Organisation'], grp_d.Year], values=grp_d['Price'] , color_continuous_scale='matter')

In [52]:
df_data.Date 

0      2020-08-07 05:12:00+00:00
1      2020-08-06 04:01:00+00:00
2      2020-08-04 23:57:00+00:00
3      2020-07-30 21:25:00+00:00
4      2020-07-30 11:50:00+00:00
                  ...           
4319   1958-02-05 07:33:00+00:00
4320   1958-02-01 03:48:00+00:00
4321   1957-12-06 16:44:00+00:00
4322   1957-11-03 02:30:00+00:00
4323   1957-10-04 19:28:00+00:00
Name: Date, Length: 4324, dtype: datetime64[ns, UTC]

# Chart the Number of Launches over Time by the Top 10 Organisations. 

How has the dominance of launches changed over time between the different players? 

In [53]:
cf = df_data.groupby('Organisation')['Date']
cf.describe() 

/usr/local/lib/python3.7/dist-packages/pandas/core/groupby/generic.py:679: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.

/usr/local/lib/python3.7/dist-packages/pandas/core/groupby/generic.py:679: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.

/usr/local/lib/python3.7/dist-packages/pandas/core/groupby/generic.py:679: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.

/usr/local/lib/python3.7/dis

,count,unique,top,freq,first,last
Organisation,,,,,,
AEB,3,3,1997-11-02 00:00:00+00:00,1,1997-11-02 00:00:00+00:00,2003-08-25 00:00:00+00:00
AMBA,8,8,1958-12-06 05:44:00+00:00,1,1958-02-01 03:48:00+00:00,1967-11-29 00:00:00+00:00
ASI,9,9,1988-03-25 19:50:00+00:00,1,1967-04-26 10:06:00+00:00,1988-03-25 19:50:00+00:00
Arianespace,279,279,1986-02-22 01:44:00+00:00,1,1984-05-23 01:33:00+00:00,2020-03-21 17:06:00+00:00
Arm??e de l'Air,4,4,1966-02-17 07:33:00+00:00,1,1965-11-26 14:47:00+00:00,1967-02-15 10:06:00+00:00
Blue Origin,12,12,2015-11-23 00:00:00+00:00,1,2015-04-29 00:00:00+00:00,2019-12-11 17:55:00+00:00
Boeing,136,136,1992-04-10 03:20:00+00:00,1,1989-02-14 18:30:00+00:00,2006-11-17 19:12:00+00:00
CASC,251,250,2008-11-05 00:15:00+00:00,2,1969-11-16 00:00:00+00:00,2020-08-06 04:01:00+00:00
CASIC,5,5,2014-11-21 06:37:00+00:00,1,2002-09-15 00:00:00+00:00,2017-03-02 23:53:00+00:00


## Create a Plotly Pie Chart comparing the total number of launches of the USSR and the USA

Hint: Remember to include former Soviet Republics like Kazakhstan when analysing the total number of launches. 

In [ ]:
chart = df_new_until_1991.groupby('locations').count()
chart

import plotly.express as px

df = chart
fig = px.pie(df, values='Location', names=df.index, title='Total number of launches of the USSR and the USA')
fig.show()

## Create a Chart that Shows the Total Number of Launches Year-On-Year by the Two Superpowers

In [ ]:
Year_On_Year = df_new_until_1991.groupby(['year', 'locations']).count() 

In [ ]:
Year_On_Year.reset_index(inplace=True)

In [ ]:
import plotly.express as px

long_df = Year_On_Year

fig = px.bar(long_df, x="year", y="Organisation", color="locations", title="Total Number of Launches Year-On-Year by the Two Superpowers",
             labels={'Organisation':'Launches', 'locations':'State', 'year':'Year'})
fig.show() 

## Chart the Total Number of Mission Failures Year on Year.

In [ ]:
df_new_until_1991 

,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status,locations,year
1718,RVSN USSR,"Site 175/58, Baikonur Cosmodrome, Kazakhstan",1991,Rokot/Briz K | GVM Demo Flight,StatusRetired,NaN,Success,Russia,1991
1719,Yuzhmash,"Site 32/2, Plesetsk Cosmodrome, Russia",1991,Tsyklon-3 | Intercosmos 25 & Magion 3,StatusRetired,NaN,Success,Russia,1991
1721,General Dynamics,"SLC-36B, Cape Canaveral AFS, Florida, USA",1991,Atlas II | Eutelsat 2F3,StatusRetired,NaN,Success,USA,1991
1722,General Dynamics,"SLC-3W, Vandenberg AFB, California, USA",1991,Atlas-E/F Star-37S-ISS | DMSP F-11,StatusRetired,NaN,Success,USA,1991
1723,RVSN USSR,"Site 133/3, Plesetsk Cosmodrome, Russia",1991,Cosmos-3M (11K65M) | Cosmos 2173,StatusRetired,NaN,Success,Russia,1991
...,...,...,...,...,...,...,...,...,...
4319,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1958,Vanguard | Vanguard TV3BU,StatusRetired,NaN,Failure,USA,1958
4320,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA",1958,Juno I | Explorer 1,StatusRetired,NaN,Success,USA,1958
4321,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1957,Vanguard | Vanguard TV3,StatusRetired,NaN,Failure,USA,1957
4322,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",1957,Sputnik 8K71PS | Sputnik-2,StatusRetired,NaN,Success,Russia,1957


In [ ]:
df_drop_false = df_new_until_1991.drop(df_new_until_1991[(df_new_until_1991['Mission_Status'] == 'Success')].index)

In [ ]:
df_drop_false = df_drop_false.groupby(['year', 'locations']).count()

In [ ]:
df_drop_false.reset_index(inplace=True)

In [ ]:
df_drop_false 

,year,locations,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status
0,1957,USA,1,1,1,1,1,0,1
1,1958,Russia,4,4,4,4,4,0,4
2,1958,USA,18,18,18,18,18,0,18
3,1959,Russia,2,2,2,2,2,0,2
4,1959,USA,10,10,10,10,10,0,10
...,...,...,...,...,...,...,...,...,...
56,1989,Russia,1,1,1,1,1,0,1
57,1990,Russia,2,2,2,2,2,0,2
58,1990,USA,1,1,1,1,1,1,1
59,1991,Russia,2,2,2,2,2,0,2


In [ ]:
import plotly.express as px

long_df = df_drop_false

fig = px.bar(long_df, x="year", y="Organisation", color="locations", title="Total Number of Mission Failures Year on Year",
             labels={'Organisation':'Failures', 'locations':'State', 'year':'Year'})
fig.show()

## Chart the Percentage of Failures over Time

Did failures go up or down over time? Did the countries get better at minimising risk and improving their chances of success over time? 

In [ ]:
percentage_of_failures = df_new_until_1991.groupby(['year', 'Mission_Status']).count()

In [ ]:
percentage_of_failures.drop(['Location', 'Date', 'Detail', 'Rocket_Status', 'Price', 'locations'], axis=1, inplace=True)

In [ ]:
perc_unstacked = percentage_of_failures.unstack("Mission_Status")


In [ ]:
perc_unstacked.reset_index(inplace=True)

In [ ]:
perc_unstacked.set_index('year', inplace=True)

In [ ]:
perc_unstacked.head(5)


Organisation                                          
Mission_Status      Failure Partial Failure Prelaunch Failure Success
year                                                                 
1957                   1.00             nan               nan    2.00
1958                  20.00            2.00               nan    6.00
1959                   9.00            3.00               nan    8.00
1960                  20.00             nan               nan   19.00
1961                  17.00            3.00               nan   32.00

In [ ]:
perc_unstacked.columns = ["_".join(pair) for pair in perc_unstacked.columns]

In [ ]:
perc_unstacked.reset_index(inplace=True)

In [ ]:
 perc_unstacked["Organisation_Failure"] = pd.to_numeric(perc_unstacked["Organisation_Failure"])
 perc_unstacked["Organisation_Partial Failure"] = pd.to_numeric(perc_unstacked["Organisation_Partial Failure"])
 perc_unstacked["Organisation_Prelaunch Failure"] = pd.to_numeric(perc_unstacked["Organisation_Prelaunch Failure"])
 perc_unstacked["Organisation_Success"] = pd.to_numeric(perc_unstacked["Organisation_Success"])

In [ ]:
perc_unstacked.fillna(0, inplace=True)

In [ ]:
perc_unstacked['failure percentage'] = perc_unstacked['Organisation_Failure'] /  (perc_unstacked['Organisation_Failure'] + perc_unstacked['Organisation_Partial Failure'] + perc_unstacked['Organisation_Prelaunch Failure'] + perc_unstacked['Organisation_Success']) *100

In [ ]:
perc_unstacked.head(2)

,year,Organisation_Failure,Organisation_Partial Failure,Organisation_Prelaunch Failure,Organisation_Success,failure percentage
0,1957,1.00,0.00,0.00,2.00,33.33
1,1958,20.00,2.00,0.00,6.00,71.43


In [ ]:
import plotly.express as px

long_df = perc_unstacked

fig = px.bar(long_df, x="year", y="failure percentage", color="failure percentage", title="Percentage of Failures over Time")
fig.show()

In [ ]:
leaders.head(2)

,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status,state
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA",2020-08-07 05:12:00+00:00,Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success,"[LC-39A, Kennedy Space Center, Florida, USA]"
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...",2020-08-06 04:01:00+00:00,Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success,"[Site 9401 (SLS-2), Jiuquan Satellite Launch ..."


In [ ]:
states = leaders['Location'].str.split(",", n=3, expand=True)

In [ ]:
states

,0,1,2,3
0,LC-39A,Kennedy Space Center,Florida,USA
1,Site 9401 (SLS-2),Jiuquan Satellite Launch Center,China,None
2,Pad A,Boca Chica,Texas,USA
3,Site 200/39,Baikonur Cosmodrome,Kazakhstan,None
4,SLC-41,Cape Canaveral AFS,Florida,USA
...,...,...,...,...
4319,LC-18A,Cape Canaveral AFS,Florida,USA
4320,LC-26A,Cape Canaveral AFS,Florida,USA
4321,LC-18A,Cape Canaveral AFS,Florida,USA
4322,Site 1/5,Baikonur Cosmodrome,Kazakhstan,None


In [ ]:
states[3].fillna(states[2], inplace=True)

In [ ]:
leaders['state'] = states[3]

# For Every Year Show which Country was in the Lead in terms of Total Number of Launches up to and including including 2020)

Do the results change if we only look at the number of successful launches? 

In [ ]:
leaders['year'] = pd.DatetimeIndex(leaders['Date']).year


In [ ]:
leaders.head(2)

,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status,state,year
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA",2020-08-07 05:12:00+00:00,Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success,USA,2020
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...",2020-08-06 04:01:00+00:00,Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success,China,2020


In [ ]:
leaders_grouped  = leaders.groupby(['year', 'state'])['Mission_Status'].count()

In [ ]:
leaders_grouped

year  state       
1957   Kazakhstan      2
       USA             1
1958   Kazakhstan      5
       USA            23
1959   Kazakhstan      4
                      ..
2020   Japan           3
       Kazakhstan      6
       New Zealand     3
       Russia          3
       USA            21
Name: Mission_Status, Length: 431, dtype: int64

In [ ]:
leaders_index = leaders_grouped.reset_index()

In [ ]:
leaders_index.set_index('year', inplace=True)


In [ ]:
leaders_index.head(20)

,year,state,Mission_Status
0,1957,Kazakhstan,2
1,1957,USA,1
2,1958,Kazakhstan,5
3,1958,USA,23
4,1959,Kazakhstan,4
5,1959,USA,16
6,1960,Kazakhstan,9
7,1960,USA,30
8,1961,Kazakhstan,7
9,1961,Russia,2


In [ ]:
import plotly.express as px

long_df = leaders_index

fig = px.bar(long_df, x="year", y="Mission_Status", color="state", title="Lead in terms of Total Number of Launches up to and including including 2020",
             labels={'Mission_Status':'Launches', 'state':'State', 'year':'Year'})
fig.show()

In [ ]:
organizations = leaders

In [ ]:
organizations

,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status,state,year
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA",2020-08-07 05:12:00+00:00,Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success,USA,2020
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...",2020-08-06 04:01:00+00:00,Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success,China,2020
2,SpaceX,"Pad A, Boca Chica, Texas, USA",2020-08-04 23:57:00+00:00,Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success,USA,2020
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan",2020-07-30 21:25:00+00:00,Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success,Kazakhstan,2020
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA",2020-07-30 11:50:00+00:00,Atlas V 541 | Perseverance,StatusActive,145.0,Success,USA,2020
...,...,...,...,...,...,...,...,...,...
4319,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1958-02-05 07:33:00+00:00,Vanguard | Vanguard TV3BU,StatusRetired,NaN,Failure,USA,1958
4320,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA",1958-02-01 03:48:00+00:00,Juno I | Explorer 1,StatusRetired,NaN,Success,USA,1958
4321,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1957-12-06 16:44:00+00:00,Vanguard | Vanguard TV3,StatusRetired,NaN,Failure,USA,1957
4322,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",1957-11-03 02:30:00+00:00,Sputnik 8K71PS | Sputnik-2,StatusRetired,NaN,Success,Kazakhstan,1957


In [ ]:
organizations_grouped = organizations.groupby(['year', 'Organisation'])['Mission_Status'].count()

In [ ]:
organizations_reset = organizations_grouped.reset_index()

In [ ]:
organizations_reset.set_index('year', inplace=True)

# Create a Year-on-Year Chart Showing the Organisation Doing the Most Number of Launches

Which organisation was dominant in the 1970s and 1980s? Which organisation was dominant in 2018, 2019 and 2020? 

In [ ]:
organizations_reset.head(10)

,Organisation,Mission_Status
year,,
1957,RVSN USSR,2
1957,US Navy,1
1958,AMBA,7
1958,NASA,2
1958,RVSN USSR,5
1958,US Air Force,2
1958,US Navy,12
1959,General Dynamics,1
1959,NASA,1


In [ ]:
result1 = organizations_reset.drop(organizations_reset.groupby(['Organisation','Mission_Status']).tail(0).index, axis=0)

In [ ]:
grouped_df = organizations_reset.groupby("year")
maximums = grouped_df.max()
maximums = maximums.reset_index()
maximums

,year,Organisation,Mission_Status
0,1957,US Navy,2
1,1958,US Navy,12
2,1959,US Navy,10
3,1960,US Air Force,21
4,1961,US Air Force,23
...,...,...,...
59,2016,VKS RF,22
60,2017,VKS RF,18
61,2018,VKS RF,37
62,2019,i-Space,27


In [ ]:
import plotly.express as px

long_df = maximums

fig = px.bar(long_df, x="year", y="Mission_Status", color="Organisation", title="Organisation Doing the Most Number of Launches")
fig.show()